### Data Stored in the Processed Output

Each processed frame contains the following keys:

| Key                        | Description |
|----------------------------|-------------|
| `CorsikaWeightMap`         | Copy of the original CORSIKA weight information. |
| `PolyplopiaPrimary`        | The primary cosmic-ray particle of the main shower. |
| `Neutrino`                 | An `I3MCTree` of neutrinos from the **main** air shower. |
| `NeutrinoCoincidence`      | An `I3MCTree` of neutrinos from **coincident** air showers. |
| `MuonTrack`                | Muon tracks from **main** air shower, extracted with `MuonGun.Track.harvest`. |
| `MuonTrackCoincidence`     | Muon tracks from **coincidence** air showers. |


In [3]:
import numpy as np
import icecube
from icecube import dataio, dataclasses, icetray, MuonGun

def files(args, include_frames=[], print_file=False):
    """A frame generator that can continue over multiple files"""
    if not isinstance(args, list):
        args = [args]
    
    for a in args:
        try:
            with dataio.I3File(a) as i3file:
                if print_file: print(f"Opening: {a}")
                for frame in i3file:
                    if len(include_frames) and not frame.Stop.id in include_frames:
                        continue
                    yield frame
        except RuntimeError:
            print(f"Error opening file: {a}")
            pass

def process_frame(frame):
    """Extract relevant particles (neutrinos + muons) from an air shower."""
    if "I3MCTree" not in frame:
        return None
    
    mctree = frame["I3MCTree"]
    new_neutrino_tree = dataclasses.I3MCTree()
    new_neutrino_coincidence_tree = dataclasses.I3MCTree()
    
    polyplopia_primary = frame["PolyplopiaPrimary"]
    
    for particle in mctree:
        if abs(particle.pdg_encoding) in [12, 14, 16]:  # Neutrinos
            if mctree.get_primary(particle) == polyplopia_primary:
                new_neutrino_tree.insert(particle)
            else:
                new_neutrino_coincidence_tree.insert(particle)
    
    muon_tracks = MuonGun.Track.harvest(frame['I3MCTree'], frame['MMCTrackList'])
    muon_tracks_primary = [mu for mu in muon_tracks if mctree.get_primary(mu) == polyplopia_primary]
    muon_tracks_coincidence = [mu for mu in muon_tracks if mctree.get_primary(mu) != polyplopia_primary]
    
    new_frame = icetray.I3Frame(icetray.I3Frame.DAQ)
    new_frame["CorsikaWeightMap"] = frame["CorsikaWeightMap"]
    new_frame["PolyplopiaPrimary"] = frame["PolyplopiaPrimary"]
    new_frame["Neutrino"] = new_neutrino_tree
    new_frame["NeutrinoCoincidence"] = new_neutrino_coincidence_tree
    #convert them into a compatible IceCube data structure
    new_frame["MuonTrack"] = dataclasses.I3VectorI3Particle(muon_tracks_primary)
    new_frame["MuonTrackCoincidence"] = dataclasses.I3VectorI3Particle(muon_tracks_coincidence)

    
    return new_frame

def process_files(input_files, output_file):
    """Process CORSIKA MC files and store selected particles in an output I3 file."""
    outfile = dataio.I3File(output_file, "w")
    
    for frame in files(input_files, print_file=True):
        new_frame = process_frame(frame)
        if new_frame:
            outfile.push(new_frame)
    
    outfile.close()
    print(f"Processed frames saved to {output_file}")



In [4]:
input_file='/data/sim/IceCube/2023/generated/CORSIKA-in-ice/22803/0000000-0000999/IC86.2023_corsika.022803.000000.i3.zst'
output_file='output_test.i3.zst'
process_files(input_file, output_file)

Opening: /data/sim/IceCube/2023/generated/CORSIKA-in-ice/22803/0000000-0000999/IC86.2023_corsika.022803.000000.i3.zst
Processed frames saved to output_test.i3.zst


In [5]:
file = dataio.I3File('output_test.i3.zst')
n=0
while file.more():
    frame=file.pop_frame()
    if 'I3MCTree' in frame:
        n=n+1
n

0

In [6]:
frame.keys()

['PolyplopiaPrimary',
 'NeutrinoCoincidence',
 'Neutrino',
 'MuonTrackCoincidence',
 'MuonTrack',
 'CorsikaWeightMap']

In [14]:
print(frame['PolyplopiaPrimary'])
print("_______________")
print(frame['Neutrino'])
print("_______________")
print(frame['NeutrinoCoincidence'])
print("_______________")
print(frame['MuonTrackCoincidence'])
print("_______________")
print(frame['MuonTrack'])
print("_______________")
print(frame['CorsikaWeightMap'])

[ I3Particle MajorID : 5015628701461841471
             MinorID : 25713402
              Zenith : 0.701966
             Azimuth : 6.14371
                   X : 93281.7
                   Y : -12854.7
                   Z : 111255
                Time : -473982
              Energy : 517067
               Speed : 0.299792
              Length : 106101
                Type : Fe56Nucleus
        PDG encoding : 1000260560
               Shape : Primary
              Status : NotSet
            Location : Anywhere
]
_______________
[I3MCTree:
  25713424 NuMu (1740.15m, -7.04928m, 1949.84m) (40.2487deg, 352.083deg) 3520.21ns 463.038GeV nanm
  25713973 NuMuBar (505.02m, 177.41m, 467.177m) (79.8808deg, 162.277deg) 11183.9ns 0.0157128GeV 0m
  25713972 NuE (505.02m, 177.41m, 467.177m) (106.006deg, 26.4295deg) 11183.9ns 0.0499454GeV 0m
  25713422 NuMuBar (1765.83m, 0.289851m, 1949.84m) (40.1963deg, 351.997deg) 3449.83ns 297.224GeV nanm
  25713956 NuEBar (624.343m, 156.117m, 597.813m) (85.9286deg